In [1]:
import json
import pickle
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt

from itertools import chain
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

### Read data

In [285]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/saved_models/comics35_pg_llama-3-8b-Instruct-bnb-4bit/comics35_results_3.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [286]:
results

{'grounds': ['{"page_utterance_emotions": [["surprise", "joy"], ["joy"], ["surprise", "joy"], ["joy"], ["joy"], ["joy"], ["surprise"], ["joy"], ["joy"], ["neutral"], ["neutral"]]}',
  '{"page_utterance_emotions": [["neutral"], ["neutral"], ["anger", "disgust"], ["anger", "disgust"], ["neutral"], ["sadness"], ["sadness"]]}',
  '{"page_utterance_emotions": [["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["fear", "surprise"], ["surprise"], ["joy"], ["anger", "surprise"], ["joy"], ["joy"], ["joy"], ["anger"], ["anger"], ["surprise", "joy"], ["fear", "sadness"], ["fear", "sadness"], ["fear", "surprise"]]}',
  '{"page_utterance_emotions": [["anger", "disgust"], ["anger", "disgust"], ["anger", "disgust"], ["fear", "sadness"], ["fear", "sadness", "surprise"], ["sadness"], ["sadness"], ["fear", "sadness"], ["sadness", "surprise"], ["sadness", "surprise"], ["joy"], ["anger"], ["anger"], ["anger"], ["anger", "disgust"], ["joy"], ["joy"], ["surprise", "joy"], ["surprise", "joy"]

In [287]:
#all_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

In [288]:
grounds = results['grounds']
predictions = results['predictions']

In [289]:
len(grounds), len(predictions)

(156, 156)

In [290]:
grounds

['{"page_utterance_emotions": [["surprise", "joy"], ["joy"], ["surprise", "joy"], ["joy"], ["joy"], ["joy"], ["surprise"], ["joy"], ["joy"], ["neutral"], ["neutral"]]}',
 '{"page_utterance_emotions": [["neutral"], ["neutral"], ["anger", "disgust"], ["anger", "disgust"], ["neutral"], ["sadness"], ["sadness"]]}',
 '{"page_utterance_emotions": [["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["fear", "surprise"], ["surprise"], ["joy"], ["anger", "surprise"], ["joy"], ["joy"], ["joy"], ["anger"], ["anger"], ["surprise", "joy"], ["fear", "sadness"], ["fear", "sadness"], ["fear", "surprise"]]}',
 '{"page_utterance_emotions": [["anger", "disgust"], ["anger", "disgust"], ["anger", "disgust"], ["fear", "sadness"], ["fear", "sadness", "surprise"], ["sadness"], ["sadness"], ["fear", "sadness"], ["sadness", "surprise"], ["sadness", "surprise"], ["joy"], ["anger"], ["anger"], ["anger"], ["anger", "disgust"], ["joy"], ["joy"], ["surprise", "joy"], ["surprise", "joy"], ["anger", "su

In [291]:
predictions

['{"page_utterance_emotions": [["joy"], ["joy"], ["joy"], ["joy"], ["joy"], ["joy"], ["surprise", "joy"], ["joy"], ["joy"], ["joy"], ["joy"]]}',
 '{"page_utterance_emotions": [["joy"], ["joy"], ["anger"], ["anger", "disgust"], ["joy"], ["anger", "disgust"], ["anger", "disgust"]]}',
 '{"page_utterance_emotions": [["anger", "disgust", "sadness"], ["anger", "disgust", "sadness"], ["anger", "disgust", "sadness"], ["fear", "surprise"], ["surprise"], ["surprise"], ["surprise"], ["surprise"], ["joy"], ["joy"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["joy"], ["joy"]]}',
 '{"page_utterance_emotions": [["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["sadness"], ["sadness"], ["sadness"], ["anger", "sadness"], ["anger", "sadness"], ["sadness"], ["sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["sadness"], ["sadness"], ["anger", "sadness"], ["sadness"], ["ang

In [292]:
grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]

In [293]:
grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [296]:
predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]

In [297]:
len(grounds), len(predictions)

(156, 156)

In [298]:
predictions

[[['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['joy'],
  ['joy'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['anger', 'disgust'],
  ['anger', 'disgust']],
 [['anger', 'disgust', 'sadness'],
  ['anger', 'disgust', 'sadness'],
  ['anger', 'disgust', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['surprise'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['joy'],
  ['joy']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['anger', 'surprise'],
  ['

In [300]:
#predictions = [x['emotions'] for x in predictions]

In [301]:
len(predictions)

156

In [302]:
len(grounds), len(predictions)

(156, 156)

In [303]:
grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [304]:
predictions

[[['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['joy'],
  ['joy'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['anger', 'disgust'],
  ['anger', 'disgust']],
 [['anger', 'disgust', 'sadness'],
  ['anger', 'disgust', 'sadness'],
  ['anger', 'disgust', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['surprise'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['joy'],
  ['joy']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['sadness'],
  ['anger', 'surprise'],
  ['

In [305]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, predictions)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

5 20 19
6 15 14
14 23 21
36 17 16
84 15 14
103 13 12
121 19 20


In [306]:
# bad_idx.sort(reverse=True)

# # Remove elements from 'grounds' at the specified indices
# for idx in bad_idx:
    
#     del grounds[idx]
#     del predictions[idx]

In [307]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]
predictions = [item for i, item in enumerate(predictions) if i not in bad_idx]

In [308]:
len(grounds), len(predictions)

(149, 149)

In [309]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in predictions for item in sublist]

In [310]:
len(grounds), len(predictions)

(1204, 1204)

In [311]:
grounds

[['surprise', 'joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['neutral'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['joy'],
 ['anger', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'sadness'],
 ['fear', 'surprise'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'sadness'],
 ['fear', 'sadness', 'surprise'],
 ['sadness'],
 ['sadness'],
 ['fear', 'sadness'],
 ['sadness', 'surprise'],
 ['sadness', 'surprise'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['surprise', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'

In [312]:
predictions

[['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'disgust', 'sadness'],
 ['anger', 'disgust', 'sadness'],
 ['anger', 'disgust', 'sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['joy'],
 ['joy'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['sadness'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['sadness'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['

In [313]:
#grounds = [item for sublist in grounds for item in sublist]
#predictions = [item for sublist in predictions for item in sublist]

In [314]:
len(grounds), len(predictions)

(1204, 1204)

In [315]:
mlb = MultiLabelBinarizer()

In [316]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

In [317]:
y_true_mhot.shape, y_pred_mhot.shape

((1204, 7), (1204, 7))

In [318]:
class_labels = mlb.classes_

In [319]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [320]:
print(classification_report(y_true_mhot, y_pred_mhot, digits=3))

              precision    recall  f1-score   support

           0      0.607     0.553     0.579       416
           1      0.206     0.171     0.187        41
           2      0.485     0.421     0.451       278
           3      0.598     0.543     0.569       276
           4      0.644     0.422     0.510        90
           5      0.527     0.645     0.580       301
           6      0.627     0.576     0.600       330

   micro avg      0.566     0.535     0.550      1732
   macro avg      0.528     0.476     0.497      1732
weighted avg      0.568     0.535     0.548      1732
 samples avg      0.594     0.559     0.549      1732



In [313]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_old_Meta-Llama-3.1-70B-Instruct-bnb-4bit/classification_report_3.pickle", 'wb') as fh:
    
    pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)

In [ ]:
# grounds = results['grounds']
# predictions = results['predictions']

In [ ]:
# grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]
# predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]
#predictions = [x['page_utterance_emotions'] for x in predictions]

In [ ]:
#grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [ ]:
#grounds_u = [[item for sublist in outer_list for item in sublist] for outer_list in grounds]

In [ ]:
#predictions_u = [[item for sublist in outer_list for item in sublist] for outer_list in predictions]

In [ ]:
#en(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
#grounds_u

[['surprise',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'neutral',
  'neutral'],
 ['neutral',
  'neutral',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'neutral',
  'sadness',
  'sadness'],
 ['anger',
  'sadness',
  'anger',
  'sadness',
  'anger',
  'sadness',
  'fear',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'joy',
  'joy',
  'joy',
  'anger',
  'anger',
  'surprise',
  'joy',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'fear',
  'surprise'],
 ['anger',
  'disgust',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'surprise',
  'sadness',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'surprise',
  'sadness',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'disgust',
  'joy',
  'joy',
  'surprise',
  'joy',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'anger',
  'surprise'],
 ['neutral', 'joy', 'joy'],
 ['neutral',
 

In [ ]:
#predictions_u

[['joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy'],
 ['neutral',
  'neutral',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'neutral',
  'anger',
  'disgust',
  'surprise',
  'anger',
  'disgust',
  'surprise'],
 ['anger',
  'anger',
  'sadness',
  'anger',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'sadness',
  'anger',
  'surprise',
  'anger',
  'joy',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'surprise',
  'joy',
  'joy'],
 ['anger',
  'anger',
  'anger',
  'sadness',
  'sadness',
  'sadness',
  'sadness',
  'anger',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'joy',
  'anger',
  'anger'],
 ['joy', 'joy', 'joy'],
 ['sadness',
  'fear',
  'surprise',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'joy',
  'joy',
  'sadness',
  'joy',
  'anger',

In [ ]:
#len(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

In [ ]:
#y_true_mhot = mlb.fit_transform(grounds_u)

In [ ]:
#y_true_mhot.shape

(156, 7)

In [ ]:
#y_true_mhot

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#y_pred_mhot = mlb.transform(predictions_u)

In [ ]:
#y_pred_mhot.shape

(156, 7)

In [ ]:
#y_pred_mhot

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#class_labels = mlb.classes_

In [ ]:
#class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [ ]:
#print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.929     0.778     0.847       117
     disgust      0.318     0.250     0.280        28
        fear      0.756     0.626     0.685        99
         joy      0.855     0.678     0.756        87
     neutral      0.778     0.311     0.444        45
     sadness      0.750     0.734     0.742        94
    surprise      0.948     0.879     0.912       124

   micro avg      0.829     0.692     0.754       594
   macro avg      0.762     0.608     0.667       594
weighted avg      0.825     0.692     0.746       594
 samples avg      0.835     0.692     0.727       594

